In [33]:
pip install langchain PyPDF2 openai

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.8 MB/s eta 0:00:00


In [16]:
!pip install transformers

from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
# Import necessary libraries for loading the local model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [2]:
!pip install langchain PyPDF2 openai
!pip install -U langchain-community
!pip install faiss-cpu
!pip install transformers

In [16]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 4.8 MB/s eta 0:00:00


In [28]:
# Step 1: Load PDF Document
def load_pdf(pdf_path):
    # Open the PDF
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        # Extract text from all pages
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [29]:
# Step 2: Create Vector Store (Modified - Treats entire text as one document)
def create_vector_store(text):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts([text], embeddings)  # Pass the entire text as a single element list
    return vector_store

In [30]:
def setup_chatbot(vector_store):
    # Create the LangChain model with a conversational retriever chain
    retriever = vector_store.as_retriever()

    # Load the local model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")  # Assuming you have downloaded the model
    model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

    # Create a pipeline for text generation
    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512  # Adjust as needed
    )

    # Use HuggingFacePipeline instead of HuggingFaceHub
    llm = HuggingFacePipeline(pipeline=pipe)

    chatbot = ConversationalRetrievalChain.from_llm(llm, retriever)
    return chatbot


In [31]:
def ask_question(chatbot, query, chat_history, full_text):  # Added full_text as input
    if query.lower() == "course title?":
        for line in full_text.split('\n'):
            if "Course Title:" in line:
                return line.split("Course Title:")[1].strip(), chat_history
    response = chatbot({"question": query, "chat_history": chat_history})
    return response['answer'], response["chat_history"]

In [32]:
if __name__ == "__main__":
    # Load PDF file (replace with your actual path)
    pdf_path = 'tech.pdf'
    full_text = load_pdf(pdf_path)  # Load the full text
    text = full_text  # Use the full text for vector store creation in this simplified case

    # Create vector store from extracted text
    vector_store = create_vector_store(text)

    # Set up the chatbot with the vector store
    chatbot = setup_chatbot(vector_store)

    # Ask a question
    query = "Course Title?"  # Example query
    chat_history = []  # Initialize chat history
    response, chat_history = ask_question(chatbot, query, chat_history, full_text)  # Pass full_text

    print("Answer:", response)

Device set to use cpu


Answer: Introduction to Data Science
